### 8장 감성 분석에 머신 러닝 적용
- 8.1 텍스트 처리용 IMDb 영화 리뷰 데이터 준비
- 8.2 BoW 모델 소개
- 8.3 문서 분류를 위한 로지스틱 회귀 모델 훈련
- 8.4 대용량 데이터 처리: 온라인 알고리즘과 외부 메모리 학습
- 8.5 잠재 디리클레 할당을 사용한 토픽 모델링
--- 

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

- 영문(IMdb)에 대한 분석과정 생략, 네이버 영화리뷰 감성분류 실습 진행 
  - ratings_train.txt / ratings_test.txt : train/test dataset, 각 15만/5만 raws, max길이 140자 
  - review_score : 부정 1~4점, 긍정 6~10점 (부정:긍정 비율 5:5) 

In [2]:
##pc에 JDK가 설치되어 있어야 함 
##!pip install konlpy 

In [3]:
df_train = pd.read_csv('data/ratings_train.txt', delimiter = '\t', keep_default_na = False)
print(df_train.shape)
df_train.head()

(150000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
df_test = pd.read_csv('data/ratings_test.txt', delimiter = '\t', keep_default_na = False)
print(df_test.shape)
df_test.head()

(50000, 3)


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [5]:
X_train = df_train['document'].values
y_train = df_train['label'].values
X_test = df_test['document'].values
y_test = df_test['label'].values

----
#### 1. konlpy의 (대표적인)형태소 분류기 
- Okt : 예전 트위터 기반 분류기, 비교적 가장 좋은 성능을 내는 것으로 알려짐 
- kkma : 띄어쓰기 오류에 덜 민감한 한글 형태소 분석기, 분석시간이 Knlpy 중에서 가장 오래 걸린다.
- Komoran : 타 형태소 분석기와 달리 공백이 포함된 형태소 단위로 분석 가능, 단 띄어쓰기 없는 문장 분석에 취약
----

In [7]:
from konlpy.tag import Okt
okt = Okt()

print(X_train[4])
print('-----------------------------------------------------------------------')
print(okt.morphs(X_train[4]))

사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
-----------------------------------------------------------------------
['사이', '몬페', '그', '의', '익살스런', '연기', '가', '돋보였던', '영화', '!', '스파이더맨', '에서', '늙어', '보이기만', '했던', '커스틴', '던스트', '가', '너무나도', '이뻐', '보였다']


---- 
#### 형태소 분석을 위한 전처리 
- ngram : NLP에서 연속된 아이템(단어,문자,기호)의 시퀀스. n=1일때 유니그램, n=2일때 바이그램 이라고도 함 
  - ex. 1-gram : 'the', 'sun', 'is', 'shining'  / 2-gram : 'the sun', 'sun is', 'is shining' 
- min_df = k : k번 미만 등장하는 토큰은 무시 
- max_df = j : j는 비중, j=0.9라면 자주 등장하는 상위 10%의 토큰은 무시 
- cf. morph 
  - nouns : 명사 추출 / morphs : 형태소 추출 / pos : 품사 부착
----

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer 
tfidf = TfidfVectorizer(ngram_range = (1,2),
                        min_df = 3, max_df = 0.9, 
                        tokenizer = okt.morphs, token_pattern = None
                       )

tfidf.fit(X_train)
X_train_okt = tfidf.transform(X_train)
X_test_okt = tfidf.transform(X_test)

--- 
#### 감성분류기 훈련 및 파라미터 튜닝 : SGDClassifier 
--- 

In [14]:
from sklearn.linear_model import SGDClassifier 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils.fixes import loguniform  ## RandomizedSearchCV 사용을 위한 탐색범위 설정을 위함 

sgd = SGDClassifier(loss = 'log', random_state = 42)
param_dist = {'alpha':loguniform(0.0001, 100.0)}  ## SGDClassifier의 매개변수 : 규제를 위한 alpha값 

rsv_okt = RandomizedSearchCV(estimator = sgd, 
                             param_distributions = param_dist, 
                             n_iter = 50, verbose = 1, random_state = 42
                             )

rsv_okt.fit(X_train_okt, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(estimator=SGDClassifier(loss='log', random_state=42),
                   n_iter=50,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001B8E6835C40>},
                   random_state=42, verbose=1)

In [16]:
print('- Train score = ', rsv_okt.score(X_train_okt, y_train))
print('- Test score = ', rsv_okt.score(X_test_okt, y_test))

- Train score =  0.83114
- Test score =  0.81426


- 약 81% 정확도의 분류기가 만들어졌음. 

--- 
   - more study needed. next book is '박조은&송영숙 저, 모두의 한국어 텍스트 분석 with Python'
--- 